In [ ]:
# 列表推导式
[i for i in range(10) if i % 2 == 0]

In [ ]:
seq = ['one','two','three']

In [ ]:
i = iter('abc')

In [ ]:
def power(values):
    for value in values:
        print('powering %s' %(value))
        yield value

def adder(values):
    for value in values:
        print('adding to %s' %(value))
        if value % 2 == 0:
            yield value + 3
        else:
            yield value + 2

elements = [1,4,7,9,12,19]
res = adder(power(elements))

In [ ]:
res.__next__()

In [ ]:
import datetime
utc = datetime.datetime.utcnow()
now = datetime.datetime.now()
'utc:{}, now:{}'.format(utc, now)

In [ ]:
import datetime
import pytz
def utcnow():
    return datetime.datetime.now(tz=pytz.utc)

print(utcnow())
print(utcnow().isoformat())
print(utcnow().isoformat() < utcnow())

In [ ]:
import iso8601
iso8601.parse_date(utcnow().isoformat())

In [ ]:
iso8601.parse_date(utcnow().isoformat()) < utcnow()

In [ ]:
utcnow()

In [ ]:
"""
函数被注册并存储在一个字段里，以便后续可以根据函数名字提取函数
"""
_functions = {}
def register(f):
    global _functions
    _functions[f.__name__] = f
    return f

@register
def foo():
    return 'bar'

In [ ]:
"""
使用functools模块中update_wrapper函数解决：新函数缺少很多原函数的属性，
如文档字符串和名字
"""
def a_decoration(func): # 函数中返回函数
    def wrap_func():
        print('before func... doing something...')
        func()
        print('after func... doing something...')
    return wrap_func

@a_decoration
def a_func():
    print('this is func...')
    
a_func()
print(a_func.__name__) # 这个时候会暴露被封装的方法

In [ ]:
from functools import wraps
def a_decoration(func): # 函数中返回函数
    @wraps(func)
    def wrap_func():
        print('before func... doing something...')
        func()
        print('after func... doing something...')
    return wrap_func

@a_decoration
def a_func():
    print('this is func...')
    
a_func()
print(a_func.__name__)

In [ ]:
"""
使用inspect.getcallargs，返回一个将参数名字和值作为键值对的字典
"""
from functools import wraps
import inspect

def a_decoration(func): # 函数中返回函数
    @wraps(func)
    def wrap_func(*args, **kwargs):
        func_args = inspect.getcallargs(func, *args, **kwargs)
        print(func_args)
        if func_args.get('username') != 'admin':
            raise Exception("This user is not allowed to get food")
        return func(*args, **kwargs)
    return wrap_func

@a_decoration
def a_func(username, type='chocolate'):
    print(type + " nom nom nom!")
    
a_func('admin','cake')

In [ ]:
class Pizza(object):
    def __init__(self, size):
        self.size = size
    def get_size(self):
        return self.size
    
# 可以向方法传入该类的任意实例，还可以传入任何对象
# 只要它包含方法期望的属性
# 每次调用类的一个方法都要对该类进行引用
# 所以python通过将类的方法绑定给实例为我们完成后续工作
# 换句话说，可以通过任何pizza访问get_size方法，
# 进一步说，python会自动将对象本身传给方法的self参数
one = Pizza(100)
Pizza.get_size(one)
m = one.get_size

# 一旦有了对绑定方法的引用则无需保持对Pizza对象的引用
# 如果有了对方法的引用，但是想知道它被绑定到了哪个对象
# 可以查看方法的__self__属性
print(m.__self__)
print(m == m.__self__.get_size)

In [68]:
# 使用abc实现抽象方法
"""
为了解决Python2&3的兼容问题，需要引入six模块，
该模块中有一个针对类的装饰器 @six.add_metaclass(MetaClass) 
可以为两个版本的Python类方便地添加metaclass
"""
import abc 
import six

@six.add_metaclass(abc.ABCMeta)
class Pizza(object):
    
    @abc.abstractmethod
    def get_radius(self):
        """Method that should do something."""
        pass
    
Pizza()

TypeError: Can't instantiate abstract class Pizza with abstract methods get_radius

In [73]:
"""
通过抽象方法使用super()
在抽象方法中会有实现代码
在Pizza中get_ingredients为累方法，并不会强迫其子类也将其定义为类方法
将其定义为静态方法也是一样，没有办法强迫子类将抽象方法实现为某种特定类型的方法
"""
import six

class Pizza(object):
    
    default_ingredients = ['cheese']
    
    @classmethod
    @abc.abstractmethod
    def get_ingredients(cls):
        """Method that should do something."""
        return cls.default_ingredients

class DietPizza(Pizza):
    def get_ingredients(self):
        return super(DietPizza, self).get_ingredients()
    
dp = DietPizza().get_ingredients()
dp

['cheese']

In [79]:
def parent():
    return object

class A(parent()):
    pass
    
A.mro() # 返回方法解析顺序用于解析属性

[__main__.A, object]

In [110]:
"""
super()函数实际上是一个构造器
每次调用都会实例化一个super对象
接受一个或两个参数，第一个参数是一个类，第二个参数是子类或第一个参数的一个实例
构造器返回的对象就像是第一个参数的父类的一个代理，它有自己的__getattribute__
方法去遍历MRO列表汇中的类并返回第一个满足条件的属性
"""

class A(object):
    bar = 42
    def foo(self):
        print("func foo...")
        pass
    
class B(object):
    bar = 0
    
class C(A, B):
    xyz = 'abc'
    
C.mro()

[__main__.C, __main__.A, __main__.B, object]

In [103]:
super(C, C()).bar

42

In [89]:
super(C, C()).foo

<bound method A.foo of <__main__.C object at 0x1038ecc50>>

In [92]:
super(B).__self__

In [93]:
super(B,B()).__self__

In [94]:
super(C) # 对象未绑定，所有不能通过它访问类属性

<super: __main__.C, None>

In [95]:
"""
super类通过某种方式实现了描述符协议(也就是__get__)，
这种方式能够让未绑定的super对象像类属性一样有用
"""
class D(C):
    sup = super(C)
    
D().sup

<super: __main__.C, <__main__.D at 0x1055664e0>>

In [96]:
D().sup.foo

<bound method A.foo of <__main__.D object at 0x105566240>>

In [97]:
D().sup.bar

42

In [108]:
super(C, C())

<super: __main__.C, <__main__.C at 0x10557ea58>>

In [111]:
class B(A):
    def foo(self):
        super().foo()

one_b = B()
one_b.foo()

func foo...
